# Wind Energy Forecasting: Inferential Statistics
The dataset chosen for this exercise is provided by Institute of Electrical and Electronics Engineers (IEEE), Power & Energy Society, and retrieved through the Kaggle database (https://www.kaggle.com/c/GEF2012-wind-forecasting).  The dataset is a time series dataset with historical power generation, wind speeds and wind directions, for the time period from July 2009 to December 2010. 

#### Importing Packages and Defining Custom Functions for Data Cleaning

In [149]:
# Import packages for data visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
# Custom Functions
def convert_to_iso(date_col):
    """
    Convert a date, originally in format YYYYMMDDHH,
    to ISO 8601 format (https://en.wikipedia.org/wiki/ISO_8601)
    
    Input: an array of DateTimes in YYYYMMDD format
    Output: an array of DateTimes in ISO 8601 format
    """
    date_col = date_col.astype(str)
    
    # year = YYYY
    year = date_col.str[0:4]
    # month = MM
    month = date_col.str[4:6]
    # day = DD
    day = date_col.str[6:8]
    # hour = HH
    hour = date_col.str[8:10]
    date_iso8601 = pd.to_datetime(year + '-' + month + '-' + day + 'T' + hour + ':00:00')
    
    return date_iso8601

def add_forecast_cat(wfn):
    """
    Add a forecast category column to the Wind Farm data
    Forecast Category 1:  1-12 hour forecasts
    Forecast Category 2: 13-24 hour forecasts
    Forecast Category 3: 25-36 hour forecasts
    Forecast Category 4: 37-48 hour forecasts
    
    Input: A DataFrame of Wind Farm data with column 'hors' containing hour-ahead forecasts 
    Output: The same DataFrame with an added column, 'forecast_cat' containing the forecast category
    """
    
    wfn['forecast_cat'] = None
    wfn.loc[ (wfn['hors'] >= 1) & (wfn['hors'] <= 12), 'forecast_cat'] = 1
    wfn.loc[ (wfn['hors'] >= 13) & (wfn['hors'] <= 24), 'forecast_cat'] = 2
    wfn.loc[ (wfn['hors'] >= 25) & (wfn['hors'] <= 36), 'forecast_cat'] = 3
    wfn.loc[ (wfn['hors'] >= 37) & (wfn['hors'] <= 48), 'forecast_cat'] = 4

    return wfn

def wfn_by_fc(wfn, forecast_cat):
    """
    Take a windfarm DataFrame and return a boolean sliced 
    version including data for a given forecast category
    
    Input: A DataFrame of Wind Farm data
    Output: The same DataFrame, but including only data for the requested forecast category
    """

    wfn = wfn.loc[(wfn['forecast_cat'] == forecast_cat)] # row slice
    return wfn

#### Importing Wind Speed and Direction Data

In [151]:
# Import wind speed and wind direction data for each wind farm, "wind farm data"
wf_dict = {'wf1': pd.read_csv('windforecasts_wf1.csv'), # Wind Farm 1
           'wf2': pd.read_csv('windforecasts_wf2.csv'), # Wind Farm 2
           'wf3': pd.read_csv('windforecasts_wf3.csv'), # Wind Farm 3
           'wf4': pd.read_csv('windforecasts_wf4.csv'), # Wind Farm 4
           'wf5': pd.read_csv('windforecasts_wf5.csv'), # Wind Farm 5
           'wf6': pd.read_csv('windforecasts_wf6.csv'), # Wind Farm 6
           'wf7': pd.read_csv('windforecasts_wf7.csv')} # Wind Farm 7

#### Importing Power Data

Note: we include only 2009-2010 data because these are the years for which there is complete data (i.e. wind speed, wind direction, and wind power data for every DateTime).

In [152]:
# Import wind power data
power = pd.read_csv('train.csv')

# Convert DateTimes to ISO 8601 format for standardization
power['date'] = convert_to_iso(power['date']) 

# Include only 2009-2010 data for wind power data
power = power.loc[ (power['date'] >= '2009-07-01') & 
                   (power['date'] <=  '2010-12-31')]

# Set index for wind power data
power.set_index('date', inplace=True)   

In [153]:
# Dictionary with wind farm data as keys and wind power data as values
wp_lookup = {'wf1':'wp1',
             'wf2':'wp2',
             'wf3':'wp3',
             'wf4':'wp4',
             'wf5':'wp5', 
             'wf6':'wp6',
             'wf7':'wp7'}

#### Cleaning Wind Speed & Direction Data, Merging with Wind Power Data
Note: we include only 2009-2010 data because these are the years for which there is complete data (i.e. wind speed, wind direction, and wind power data for every DateTime).

In [154]:
for key, _ in wf_dict.items():
    
    # Convert date-times to ISO 8601 format for standardization
    wf_dict[key]['date'] = convert_to_iso(wf_dict[key]['date'])
    # Initialize mod_date column
    wf_dict[key]['mod_date'] = (wf_dict[key]['date'] + 
                                pd.to_timedelta(arg=wf_dict[key]['hors'],unit='h'))
    # Initialize forecast_cat column
    wf_dict[key] = add_forecast_cat(wf_dict[key])

    # Include only 2009-2010 data for wind speed/direction data
    wf_dict[key] = wf_dict[key].loc[(wf_dict[key]['mod_date'] >= '2009-07-01') & 
                                    (wf_dict[key]['mod_date'] <= '2010-12-31')]
    # Set Index column
    wf_dict[key].set_index('mod_date',inplace=True)
    
    # Merge wind speed/direction data with wind power data
    wf_dict[key] = wf_dict[key].merge(power[[wp_lookup[key]]], 
                                      how='left',
                                      left_index=True,       
                                      right_index=True)

In [155]:
# Explore wind farm data
print(wf_dict['wf1'].head(15))

                                   date  hors     u     v    ws      wd  \
2009-07-01 01:00:00 2009-07-01 00:00:00     1  2.34 -0.79  2.47  108.68   
2009-07-01 02:00:00 2009-07-01 00:00:00     2  2.18 -0.99  2.40  114.31   
2009-07-01 03:00:00 2009-07-01 00:00:00     3  2.20 -1.21  2.51  118.71   
2009-07-01 04:00:00 2009-07-01 00:00:00     4  2.35 -1.40  2.73  120.86   
2009-07-01 05:00:00 2009-07-01 00:00:00     5  2.53 -1.47  2.93  120.13   
2009-07-01 06:00:00 2009-07-01 00:00:00     6  2.66 -1.29  2.96  115.79   
2009-07-01 07:00:00 2009-07-01 00:00:00     7  2.69 -0.81  2.81  106.71   
2009-07-01 08:00:00 2009-07-01 00:00:00     8  2.72 -0.26  2.73   95.39   
2009-07-01 09:00:00 2009-07-01 00:00:00     9  2.87  0.08  2.87   88.50   
2009-07-01 10:00:00 2009-07-01 00:00:00    10  3.23 -0.01  3.23   90.19   
2009-07-01 11:00:00 2009-07-01 00:00:00    11  3.65 -0.33  3.66   95.15   
2009-07-01 12:00:00 2009-07-01 00:00:00    12  3.89 -0.60  3.94   98.71   
2009-07-01 13:00:00 2009-

NOTE: All data is unitless.
**Description:** Index is dates in ISO-8601 DateTime format. 'date' column data is unformatted DateTime data. 'hors' column data is unformatted hour data, representing the number of hours-ahead being forecasted at the corresponding DateTime. 'u' is magnitude of x-axis wind speed vector. 'v' is magnitude of y-axis wind speed vector. 'ws' is magnitude of wind speed. 'wd' is angle of wind direction. 'forecast_cat' is forecast category, ranging from 1-4.
'wp(n)' column represents wind power data where n=1...7 represent each of the seven wind farms.

### Inferential Statistics

There are four forecast categories-- Forecast Category 1: 1-12 hours ahead. Forecast Category 2: 13-24 hours ahead. Forecast Category 3: 25-36 hours ahead. Forecast Category 4: 37-48 hours ahead.

We are interested to see if the wind speeds of one forecast category different from the wind speeds of any other forecast category. 

As an example, we could take a look at Wind Farm 1, and compare the mean wind speed of Forecast Category 1 ($\mu$<sub>1</sub>) to the mean wind speed of Forecast Category 2-4 ($\mu$<sub>234</sub>). The null hypothesis in this case would be that the mean wind speed of Forecast Category 1 are equal to the mean wind speed of Forecast Category 2-4: 

$\mu$<sub>1</sub> = $\mu$<sub>234</sub>

The alternate hypothesis would then be that the mean wind speed of Forecast Category 1 is not equal to the mean wind speed of the other Forecast Categories: 

$\mu$<sub>1</sub> $\neq$ $\mu$<sub>234</sub>

In [156]:
# Subset Wind Farm 1 wind speeds into forecast categories
fc1 = wfn_by_fc(wf_dict['wf1'], 1)['ws']
fc2 = wfn_by_fc(wf_dict['wf1'], 2)['ws']
fc3 = wfn_by_fc(wf_dict['wf1'], 3)['ws']
fc4 = wfn_by_fc(wf_dict['wf1'], 4)['ws']
fc_all = ((fc1.append(fc2)).append(fc3)).append(fc4)

In [157]:
# Combine wind speeds of forecast categories 2, 3, and 4
fc234 = (fc2.append(fc3)).append(fc4)

# Compute mean wind speed of Forecast Category 1: m1
m1 = np.mean(fc1)
# Compute mean wind speed of Forecast Category 2, 3, 4: m234
m234 = np.mean(fc234)
# Compute mean wind speed of all Forecast Categories: m_all
m_all = np.mean(fc_all)

# Compute Empirical Difference in Mean Wind Speeds: empirical_diff_means
empirical_diff_means = m1 - m234

# Generate Shifted Arrays
fc1_shifted = fc1 - m1 + m_all
fc234_shifted = fc234 - m234 + m_all

# Compute 1000 bootstrap replicates from shifted arrays
size = 1000
bs_replicates_1 = np.empty(size)
bs_replicates_234 = np.empty(size)
for i in range(size):
    bs_replicates_1[i] = np.mean(np.random.choice(fc1_shifted,len(fc1_shifted)))
    bs_replicates_234[i] = np.mean(np.random.choice(fc234_shifted,len(fc234_shifted)))

# Compute difference of means: bs_replicates
bs_replicates = bs_replicates_1 - bs_replicates_234

# Compute and print p-value: p
p = np.sum(bs_replicates == empirical_diff_means) / size
print('Null Hypothesis: m1 = m234')
print('p-value, bootstrap approach: {:5f}', p)

# Compute the 95% confidence interval, bootstrap approach
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Compute Margin of Error, Bootstrap Replicate Approach
margin_of_error = 1.96 * bs_replicates.std()
print('The 95% confidence interval is from ', conf_int[0], ' to ', conf_int[1], '.')
print('The margin of error, as computed from the bootstrap replicates, is: ', margin_of_error)

Null Hypothesis: m1 = m234
p-value, bootstrap approach: {:5f} 0.0
The 95% confidence interval is from  -0.038797852705  to  0.038106618047 .
The margin of error, as computed from the bootstrap replicates, is:  0.0391359191669


##### Comparing Forecast Category 2 with Forecast Category 1, 3, 4

Similarly, we could also compare the mean wind speed of Forecast Category 2, $\mu$<sub>2</sub>, to the remaining Forecast Categories, $\mu$<sub>134</sub>.

Null Hypothesis: 

$\mu$<sub>2</sub> = $\mu$<sub>134</sub>

The alternate hypothesis would then be that the mean wind speed of Forecast Category 2 is not equal to the mean wind speed of the other Forecast Categories: 

$\mu$<sub>2</sub> $\neq$ $\mu$<sub>134</sub> 

In [158]:
# Combine wind speeds of forecast categories 1, 3, and 4
fc134 = (fc1.append(fc3)).append(fc4)

# Compute mean wind speed of Forecast Category 1: m2
m2 = np.mean(fc2)
# Compute mean wind speed of Forecast Category 1, 3, 4: m134
m134 = np.mean(fc134)

# Compute Empirical Difference in Mean Wind Speeds: empirical_diff_means
empirical_diff_means = m2 - m134

# Generate Shifted Arrays
fc2_shifted = fc2 - m2 + m_all
fc134_shifted = fc234 - m134 + m_all

# Compute 1000 bootstrap replicates from shifted arrays
size = 1000
bs_replicates_2 = np.empty(size)
bs_replicates_134 = np.empty(size)
for i in range(size):
    bs_replicates_2[i] = np.mean(np.random.choice(fc2_shifted,len(fc2_shifted)))
    bs_replicates_134[i] = np.mean(np.random.choice(fc134_shifted,len(fc134_shifted)))

# Compute difference of means: bs_replicates
bs_replicates = bs_replicates_2 - bs_replicates_134

# Compute and print p-value: p
p = np.sum(bs_replicates == empirical_diff_means) / size
print('Null Hypothesis: m2 = m134')
print('p-value, bootstrap approach: {:5f}', p)

# Compute the 95% confidence interval, bootstrap approach
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Compute Margin of Error, Bootstrap Replicate Approach
margin_of_error = 1.96 * bs_replicates.std()
print('The 95% confidence interval is from ', conf_int[0], ' to ', conf_int[1], '.')
print('The margin of error, as computed from the bootstrap replicates, is: ', margin_of_error)

Null Hypothesis: m2 = m134
p-value, bootstrap approach: {:5f} 0.0
The 95% confidence interval is from  -0.0257981584471  to  0.0450181974404 .
The margin of error, as computed from the bootstrap replicates, is:  0.0362249881835


##### Comparing Forecast Category 3 with Forecast Category 1, 2, 4

Similarly, we could also compare the mean wind speed of Forecast Category 3, $\mu$<sub>3</sub>, to the remaining Forecast Categories, $\mu$<sub>124</sub>.

$\mu$<sub>3</sub> = $\mu$<sub>124</sub>

The alternate hypothesis would then be that the mean wind speed of Forecast Category 3 is not equal to the mean wind speed of the other Forecast Categories: 

$\mu$<sub>3</sub> $\neq$ $\mu$<sub>124</sub>

In [161]:
# Combine wind speeds of forecast categories 1, 2, and 4
fc124 = (fc1.append(fc2)).append(fc4)

# Compute mean wind speed of Forecast Category 1: m3
m3 = np.mean(fc3)
# Compute mean wind speed of Forecast Category 1, 2, 4: m124
m124 = np.mean(fc124)

# Compute Empirical Difference in Mean Wind Speeds: empirical_diff_means
empirical_diff_means = m3 - m124

# Generate Shifted Arrays
fc3_shifted = fc3 - m3 + m_all
fc124_shifted = fc124 - m124 + m_all

# Compute 1000 bootstrap replicates from shifted arrays
size = 1000
bs_replicates_3 = np.empty(size)
bs_replicates_124 = np.empty(size)
for i in range(size):
    bs_replicates_3[i] = np.mean(np.random.choice(fc3_shifted,len(fc3_shifted)))
    bs_replicates_124[i] = np.mean(np.random.choice(fc124_shifted,len(fc124_shifted)))

# Compute difference of means: bs_replicates
bs_replicates = bs_replicates_3 - bs_replicates_124

# Compute and print p-value: p
p = np.sum(bs_replicates == empirical_diff_means) / size
print('Null Hypothesis: m3 = m124')
print('p-value, bootstrap approach: {:5f}', p)

# Compute the 95% confidence interval, bootstrap approach
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Compute Margin of Error, Bootstrap Replicate Approach
margin_of_error = 1.96 * bs_replicates.std()
print('The 95% confidence interval is from ', conf_int[0], ' to ', conf_int[1], '.')
print('The margin of error, as computed from the bootstrap replicates, is: ', margin_of_error)

Null Hypothesis: m3 = m124
p-value, bootstrap approach: {:5f} 0.0
The 95% confidence interval is from  -0.0372781372494  to  0.0405696912685 .
The margin of error, as computed from the bootstrap replicates, is:  0.0386960758443


##### Comparing Forecast Category 4 with Forecast Category 1, 2, 3

Final, we can compare the mean wind speed of Forecast Category 4, $\mu$<sub>4</sub>, to the remaining Forecast Categories, $\mu$<sub>123</sub>.

$\mu$<sub>4</sub> = $\mu$<sub>123</sub>

The alternate hypothesis would then be that the mean wind speed of Forecast Category 4 is not equal to the mean wind speed of the other Forecast Categories: 

$\mu$<sub>4</sub> $\neq$ $\mu$<sub>123</sub>

In [160]:
# Combine wind speeds of Forecast Category 1, 2, 3
fc123 = (fc1.append(fc2)).append(fc3)

# Compute mean wind speed of Forecast Category 1: m4
m4 = np.mean(fc4)
# Compute mean wind speed of Forecast Category 1, 2, 3: m123
m123 = np.mean(fc123)

# Compute Empirical Difference in Mean Wind Speeds: empirical_diff_means
empirical_diff_means = m4 - m123

# Generate Shifted Arrays
fc4_shifted = fc4 - m4 + m_all
fc123_shifted = fc123 - m123 + m_all

# Compute 1000 bootstrap replicates from shifted arrays
size = 1000
bs_replicates_4 = np.empty(size)
bs_replicates_123 = np.empty(size)
for i in range(size):
    bs_replicates_4[i] = np.mean(np.random.choice(fc4_shifted,len(fc4_shifted)))
    bs_replicates_123[i] = np.mean(np.random.choice(fc123_shifted,len(fc123_shifted)))

# Compute difference of means: bs_replicates
bs_replicates = bs_replicates_4 - bs_replicates_123

# Compute and print p-value: p
p = np.sum(bs_replicates == empirical_diff_means) / size
print('p-value, bootstrap approach: {:5f}', p)

# Compute the 95% confidence interval, bootstrap approach
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Compute Margin of Error, Bootstrap Replicate Approach
margin_of_error = 1.96 * bs_replicates.std()
print('The 95% confidence interval is from ', conf_int[0], ' to ', conf_int[1], '.')
print('The margin of error, as computed from the bootstrap replicates, is: ', margin_of_error)

p-value, bootstrap approach: {:5f} 0.0
The 95% confidence interval is from  -0.0376638138182  to  0.0382324758983 .
The margin of error, as computed from the bootstrap replicates, is:  0.0380381460775


### Conclusion

We used bootstrapping techniques to compare the mean wind speed of Forecast Category 1 to that of the other Forecast Categories, and repeated the process for Forecast Category 2, Forecast Category 3 and Forecast Category 4.  All statistics returned p-values of 0, such that we could reject the null hypothesis that the wind speeds of different forecast categories are equal.

These preliminary inferential statistics on wind speed data from wind farm 1 indicate that wind speeds of different forecast categories are significantly different from each other. This is as we would expect. If the wind speeds of different forecast categories were the same, this would indicate that there is redundancy in forecasting, and this may be reason to spend less resources on forecasting.